In [ ]:
import mysql.connector
from datetime import date

# Hàm kết nối database
def connect_to_db():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='yourpassword',
        database='mytinylibrary'
    )

# Hàm gọi stored procedure quản lí việc mượn sách
def borrow_book(reader_id, book_id, staff_id, borrow_date):
    try:
        connection = connect_to_db()
        cursor = connection.cursor()

        args = [reader_id, book_id, staff_id, borrow_date, ""]
        result = cursor.callproc("borrow_book", args)

        print(result[4])

        connection.commit()
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

# Hàm gọi stored procedure quản lí việc trả sách
def return_book(borrow_id, return_staff_id, return_date):
    try:
        connection = connect_to_db()
        cursor = connection.cursor()

        args = [borrow_id, return_staff_id, return_date, ""]
        result = cursor.callproc("return_book", args)

        print(result[3])

        connection.commit()
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

# Hàm tổng quát gọi các stored procedure khác
def call_procedure(proc_name, params=()):
    try:
        conn = connect_to_db()
        cursor = conn.cursor()

        cursor.callproc(proc_name, params)

        for result in cursor.stored_results():
            rows = result.fetchall()
            if rows:
                print(f"\nResult from `{proc_name}`:")
                for row in rows:
                    print(row)
            else:
                print(f"\nProcedure `{proc_name}` returned no data.")

        conn.commit()
    except mysql.connector.Error as err:
        print(f"Error calling `{proc_name}`: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()


# Test
if __name__ == "__main__":
    # Stored Procedure quản lí mượn sách
    borrow_book(1, 3, 2, date.today())

    # Stored Procedure quản lí trả sách
    return_book(18, 5, date.today())

    # Gọi các stored procedure khác
    call_procedure("overdue_report")
    call_procedure("get_all_books")
    call_procedure("get_book_by_id", [5])
    call_procedure("get_reader_by_id", [2])
    call_procedure("get_borrowings_of_readerid", [5])
    call_procedure("get_borrowings_of_bookid", [5])
    call_procedure("get_currently_borrowed_books")